In [3]:
using Pkg
Pkg.activate(".")
#Pkg.add("Turing")

  Activating new project at `d:\Code\ProbabilisticProgramming`
    Updating registry at `C:\Users\fuerf\.julia\registries\General.toml`
   Resolving package versions...
   Installed OffsetArrays ────────────── v1.14.1
   Installed StatsFuns ───────────────── v1.3.2
   Installed Accessors ───────────────── v0.1.37
   Installed AbstractMCMC ────────────── v5.3.0
   Installed MutableArithmetics ──────── v1.4.6
   Installed DSP ─────────────────────── v0.7.9
   Installed StatisticalTraits ───────── v3.4.0
   Installed TimerOutputs ────────────── v0.5.24
   Installed HypergeometricFunctions ─── v0.3.24
   Installed StaticArrays ────────────── v1.9.7
   Installed DomainSets ──────────────── v0.7.14
   Installed ProgressMeter ───────────── v1.10.2
   Installed Turing ──────────────────── v0.34.1
   Installed NNlib ───────────────────── v0.9.23
   Installed IntelOpenMP_jll ─────────── v2024.2.1+0
   Installed Roots ───────────────────── v2.1.8
   Installed RecursiveArrayTools ─────── v3.27.0
 

In [6]:
using Turing, Distributions, Statistics

In [2]:
mu = 1
sigma = 2
n_contributers = 2
n_samples = 1000

Na = Normal(mu, sigma)
Nb = Normal(1, 1)
Nc = Normal(-1, 1)

Normal{Float64}(μ=-1.0, σ=1.0)

In [3]:
contributers = rand(1:n_contributers, n_samples)
data = rand(Na, n_samples) + (contributers .== 1) .* rand(Nb, n_samples)+ (contributers .== 2) .* rand(Nc, n_samples)

1000-element Vector{Float64}:
 -1.0470814064569125
  5.719624363280163
 -2.5107223835032006
 -1.6536134012163248
  0.7347656252294162
  0.17319608278119314
  0.18775989351129807
  2.6575761367003388
  2.476778975286964
  0.947304140460199
  ⋮
  1.8423346561047844
  3.126623475423444
  2.249755530845948
  4.334786821822841
  0.7509229172653464
  1.1842848389410445
 -0.24161906414249845
  4.34512253614721
  1.8147769132097284

In [10]:
@model function model_fun(data, contributers)
    sigmax = 5
    μ_a ~ Normal( 0, 10)
    pb_sig ~ Uniform(0,1)
    pa_sig ~ Uniform(0,1)

    sigma = sqrt(pa_sig * sigmax)
    sigma_b = sqrt(pb_sig * sigmax * (1-pa_sig) )
    sigma_c = sqrt(sigmax - sigma - sigma_b)

    μ_b ~ Normal(0, 10)

    b ~ Normal.(μ_b, sigma_b)
    c ~ Normal.(-μ_b, sigma_c)
    
    m = μ_a .+ (contributers .== 1) .* b .+ (contributers .== 2) .* c

    data .~ Normal.(m, sigma)
end #--> works :-D

@model function model_fun2(data, contributers)
    sigmax = var(data)
    μ_a ~ Normal( 0, 10)
    sigmax ~ Uniform(0, sigmax)
    #=
    pb_sig ~ Uniform(0,1)
    pa_sig ~ Uniform(0,1)

    sigma = sqrt(pa_sig * sigmax)
    sigma_b = sqrt(pb_sig * sigmax * (1-pa_sig) )
    sigma_c = sqrt(sigmax - sigma - sigma_b)
    =#
    μ_b ~ Normal(0, 10)

    b ~ Normal.(μ_b, 2)
    c ~ Normal.(-μ_b, 2)
    
    m = μ_a .+ (contributers .== 1) .* b .+ (contributers .== 2) .* c

    data .~ Normal.(m, sigma)
end #--> works :-D


model_fun2 (generic function with 2 methods)

In [11]:
model = model_fun2(data, contributers)
chain = sample(model, NUTS(), 2000)

Sampling   0%|█                                         |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.2
└ @ Turing.Inference C:\Users\fuerf\.julia\packages\Turing\QN7BL\src\mcmc\hmc.jl:212
Sampling   0%|█                                         |  ETA: 0:21:42
Sampling   1%|█                                         |  ETA: 0:11:04
Sampling   2%|█                                         |  ETA: 0:07:29
Sampling   2%|█                                         |  ETA: 0:05:43
Sampling   2%|██                                        |  ETA: 0:04:38
Sampling   3%|██                                        |  ETA: 0:03:54
Sampling   4%|██                                        |  ETA: 0:03:22
Sampling   4%|██                                        |  ETA: 0:03:00
Sampling   4%|██                                        |  ETA: 0:02:41
Sampling   5%|███                                       |  ETA: 0:02:26
Sampling   6%|███                                       |  ETA: 0:02:15
Sampling   

Chains MCMC chain (2000×17×1 Array{Float64, 3}):

Iterations        = 1001:1:3000
Number of chains  = 1
Samples per chain = 2000
Wall duration     = 30.68 seconds
Compute duration  = 30.68 seconds
parameters        = μ_a, sigmax, μ_b, b, c
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

         μ_a    1.0683    1.4438    0.0656    487.8816    782.9003    0.9997   ⋯
      sigmax    2.9949    1.7388    0.0457   1386.9426   1194.8395    1.0007   ⋯
         μ_b    0.9624    1.4385    0.0419   1177.5160   1062.2220    0.9998   ⋯
           b    0.9196    1.4424    0.0659    483.0411    811.5192    0.9999   ⋯
           c   -1.0618    1.4443    0.0657    486

In [34]:
@model function model_fun3(data, contributers)
    n_contributers = maximum(contributers)
    sigmax = var(data)
    max_con = 5 #Maximum expected value for contributers
    μ_a ~ Normal( 0, 10) 
    sigmax ~ Uniform(0, sigmax)

    μ_cons = Vector{Float16}(undef, n_contributers)

    cons = Vector{Float16}(undef, n_contributers)

    for i in 1:n_contributers-1
        μ_cons[i] ~ Uniform(0, max_con)
        cons[i] ~ Normal(μ_cons[i], sigmax)
    end
    μ_cons[n_contributers] ~ Uniform(0, max_con)
    cons[n_contributers] ~ Normal(-sum(μ_cons[1:n_contributers-1]), sigmax) #normalization
    
    m = μ_a
    for con in 1: n_contributers
        m += (contributers .== con) .* cons[con]
    end
    data .~ Normal.(m, sigma)
end

model_fun3 (generic function with 4 methods)

In [35]:
model = model_fun3(data, contributers)
chain = sample(model, NUTS(), 2000)

MethodError: MethodError: no method matching +(::Float64, ::Vector{Float64})
For element-wise addition, use broadcasting with dot syntax: scalar .+ array

Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:587
  +(::AbstractFloat, !Matched::ForwardDiff.Dual{Ty}) where Ty
   @ ForwardDiff C:\Users\fuerf\.julia\packages\ForwardDiff\PcZ48\src\dual.jl:145
  +(::Number, !Matched::Symbolics.Num)
   @ Symbolics C:\Users\fuerf\.julia\packages\SymbolicUtils\t2eAT\src\methods.jl:76
  ...


In [23]:
maximum(contributers)

2